In [7]:
import segmentation_models as sm

sm.set_framework('tf.keras')

from cnn.input import PascalVOC2012Dataset, Dataloader, get_training_augmentation

import os

In [8]:
train_dataset_descriptor_filepath = os.path.join(
    'pascalvoc12',
    'VOCdevkit',
    'VOC2012',
    'ImageSets',
    'Segmentation',
    'train.txt',
)

val_dataset_descriptor_filepath = os.path.join(
    'pascalvoc12',
    'VOCdevkit',
    'VOC2012',
    'ImageSets',
    'Segmentation',
    'val.txt',
)

images_path = os.path.join(
    'pascalvoc12', 
    'VOCdevkit', 
    'VOC2012', 
    'JPEGImages'
)

masks_path = os.path.join(
    'pascalvoc12', 
    'VOCdevkit', 
    'VOC2012', 
    'SegmentationClass'
) 

In [9]:
image_height = 256
image_width = 256

In [10]:
train_dataset = PascalVOC2012Dataset(
    train_dataset_descriptor_filepath,
    images_path=images_path,
    masks_path=masks_path,
    image_height=image_height,
    image_width=image_width,
    augmentation=get_training_augmentation(image_height, image_width),
)

val_dataset = PascalVOC2012Dataset(
    val_dataset_descriptor_filepath,
    images_path=images_path,
    masks_path=masks_path,
    image_height=image_height,
    image_width=image_width,
)

In [11]:
train_dataloader = Dataloader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = Dataloader(val_dataset, batch_size=16, shuffle=False)

In [12]:
net = sm.Unet('resnet34', classes=21, input_shape=(image_height, image_width, 3), activation='softmax', encoder_weights='imagenet')

net.compile(                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
    optimizer='Adam',
    loss=sm.losses.categorical_focal_dice_loss,
    metrics=[sm.metrics.IOUScore(threshold=0.5)],
)

history = net.fit(train_dataloader, 
                validation_data=val_dataloader,
                epochs=50)


Epoch 1/50


: 

: 